In [0]:
!pip install h2o

    100% |████████████████████████████████| 120.9MB 289kB/s 
  Stored in directory: /root/.cache/pip/wheels/f4/21/59/e953dbf8206d5061ce1346e119dad9ffafd7f88011430682ba
Successfully built h2o


In [0]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
import psutil
import pandas as pd
import numpy as np
#from tpot import TPOTClassifier

#from autosklearn import  classification

Define the configuration parameters and initialize the H2O

In [0]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

6


In [0]:
h2o.init(strict_version_check=False,max_mem_size=min_mem_size)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpljdlo5qh
  JVM stdout: /tmp/tmpljdlo5qh/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpljdlo5qh/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,"14 days, 21 hours and 19 minutes"
H2O cluster name:,H2O_from_python_unknownUser_w5xwe1
H2O cluster total nodes:,1
H2O cluster free memory:,6 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
output = pd.read_csv('API_dataset.csv')

In [0]:
y='label'

In [0]:
def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ] ## add aws colums too
    # determine column types
    ints, reals= [], []
    for key, val in df.dtypes.items():
        if key in C:
            if val == 'int':
                ints.append(key)           
            else: 
                reals.append(key)    
    x=ints+reals
    return x

In [0]:
hf = h2o.H2OFrame(output)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
x=get_independent_variables(output,y)
x.pop()
x.pop()
x.pop()
x

['Text', 'IbmScore', 'GCPScore', 'AzureScore']

In [0]:
run_time=500
run_id='SOME_ID_20180617_221529' # Just some arbitrary ID
server_path='/Users/bear/Documents/INFO_7390/H2O'
classification=True
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="automl_test"  # project_name = project

In [0]:
aml = H2OAutoML(max_runtime_secs=run_time,project_name = project,balance_classes=balance_y)

In [0]:
training, validation = hf.split_frame(ratios = [.8], seed = 11)

In [0]:
aml.train(x=x,
          y=y,
          training_frame=training)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [0]:
best_model = h2o.get_model(aml.leaderboard[0,'model_id'])

In [0]:
best_model.algo

'drf'

In [0]:
aml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
XRT_1_AutoML_20190328_174759,0.525599,3.18704,0.552657,0.30543
GBM_4_AutoML_20190328_174759,0.529909,0.883459,0.544976,0.296999
DRF_1_AutoML_20190328_174759,0.530412,2.94996,0.548577,0.300936
XGBoost_1_AutoML_20190328_174759,0.53131,0.808189,0.54466,0.296655
GBM_2_AutoML_20190328_174759,0.533463,0.846314,0.541566,0.293294
XGBoost_grid_1_AutoML_20190328_175500_model_2,0.536422,1.0767,0.659189,0.43453
XGBoost_3_AutoML_20190328_174759,0.536778,0.845723,0.563309,0.317317
GBM_grid_1_AutoML_20190328_174759_model_12,0.538307,0.987466,0.625604,0.391381
XGBoost_grid_1_AutoML_20190328_174759_model_2,0.539836,0.96631,0.617307,0.381068
GBM_grid_1_AutoML_20190328_174759_model_1,0.53991,1.03088,0.64263,0.412973


In [0]:
#xgb_train, xgb_test= hf.split_frame([0.8],seed=12345)

In [0]:
predictions=aml.leader.predict(validation)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [0]:
#del hf_predict_values
#hf_predict_values= h2o.H2OFrame()
#predictions = XGB_model.predict(xgb_test)
predictions['Text']= validation['Text']

In [0]:
predictions.head(10)

predict,negative,neutral,positive,Text
neutral,0,0.645161,0.354839,"Moving on to debt Investing & Lending on Page 10. In the fourth quarter, net revenues from debt securities and loans were $912 million, largely driven by net interest income of roughly $800 million. For the full year, debt Investing & Lending generated $3.8 billion of net revenues, including net interest income of approximately $2.7 billion. I would point out that before accounting for future growth, we begin 2019 with an annualized net interest income run rate of $3.2 billion. Our debt Investing & Lending balance sheet, as shown on the lower left of Page 10, ended the quarter at $113 billion, which includes $94 billion in loans and $11 billion in debt securities. Our lending is franchise adjacent and continues to remain conservative with approximately 85% of our loan portfolio secured as of year-end. This quarter, we took provision for loan losses of $222 million, reflecting loan growth."
neutral,0,0.998811,0.00118906,Michael Mayo - Wells Fargo Securities
neutral,0.111008,0.888067,0.000925069,Glenn Schorr
neutral,0,1,0,Your next question is from the line of Michael Carrier with Bank of America Merrill Lynch.
positive,0.156647,0,0.843353,"Sure. Thanks for the question. I think it's difficult for me to give you a sense of the delta in the [Technical Difficulty] relative to where we carry it. I would ask you to sort of rely more on the progression and the stability and sustainability of this business just in terms of what it has produced for many, many quarters over several years in the context of the investing. And I think that the strength of that franchise and the strength of the investing is sort of an adjacent thread that runs across the whole of the firm, which is that the Investment Banking franchise is an extraordinary sourcing engine off of which we look to make these investments. And we try to find opportunities where there is considerable opportunity for growth. What I sense in your question is the sort of distinction, if you will, between realized and unrealized gain. Part of what I wanted to point out in answering the prior question is that there's a good deal here of realized gain in the book and a continual velocity turn in the portfolio, and sales or partial sales sort of are one part of that. And then looking at the performance of the business is quite another. David and I have committed ourselves, and I've said this now several times, that our ambition is to put more light into the I&L segment, just given what it produces and how it is a major component piece. I know that you asked on the equity side. Let me just spend 2 seconds on the debt side of I&L, which I think has even greater durability and predictability to it. So this is where we are making debt investments or expanding credit in and around the adjacencies of the firm, whether it's the corporates or others who are themselves clients of the firm. We're doing that high up in the capital structure, and we've developed a portfolio that is about 85% secured."
neutral,0.28638,0.391472,0.322148,"Well, we need to bring the Shanghai factory online. I think that's the biggest driver for getting to 500K plus a year. Our car is just very expensive going into China. We've got import duties. We've got transport costs. We've got higher-cost labor here. And we've never been eligible for any of the EV tax credits. A lot of people sort of dependent on incentives. In fact, we are [indiscernible] EVs, we have the least access to incentives. It's pretty crazy because there's so many companies that - countries that have put price caps on the EV incentive, which affects Tesla. And in China, which is the biggest market for EVs, we've never had any subsidies or tax incentives for vehicles."
positive,0.0230355,0.273259,0.703706,"So it's - it is eligible for that. But it sounds like that's going to be reducing in China in the coming years. But, really, bottom line is, we need the Shanghai factory to achieve tha

In [0]:
h2o.export_file(predictions,path = "H2O_result.csv", force=True, parts=1)

Export File progress: |███████████████████████████████████████████████████| 100%


In [0]:
confMetrics = aml.leader.model_performance(validation)
confMetrics.confusion_matrix()


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
12.0,10.0,12.0,0.6470588,22 / 34
0.0,127.0,34.0,0.2111801,34 / 161
1.0,21.0,108.0,0.1692308,22 / 130
13.0,158.0,154.0,0.24,78 / 325
